# Fine-Tune Qwen2.5-7B for Lexmancer Ability Generation

This notebook fine-tunes Qwen2.5-7B-Instruct on ability JSON generation using LoRA adapters.

**Requirements:**
- Google Colab with GPU (T4 free tier works, A100 recommended)
- Training data in JSONL format (instruction/input/output)

**Output:**
- Fine-tuned model merged and quantized to GGUF format
- Ready to replace `Assets/LLM/qwen2.5-7b-instruct-q4_k_m.gguf`

## Step 1: Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install Unsloth for fast fine-tuning
%%capture
!pip install unsloth
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# Install additional dependencies
%%capture
!pip install datasets trl peft transformers accelerate bitsandbytes

## Step 2: Load Base Model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Qwen2.5 supports up to 32K, but 2K is enough for abilities
dtype = None  # Auto-detect (Float16 for Tesla T4, V100, Bfloat16 for A100)
load_in_4bit = True  # Use 4bit quantization for memory efficiency

# Load Qwen2.5-7B-Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # trust_remote_code=True,  # Uncomment if needed
)

print("✅ Model loaded successfully!")

## Step 3: Add LoRA Adapters

In [ ]:
# Add LoRA adapters for parameter-efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank (higher = more parameters, 8-64 typical)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,  # LoRA scaling factor (usually same as r)
    lora_dropout=0,  # Dropout (0 = no dropout, Unsloth optimizes this)
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory efficient
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA adapters added!")

## Step 4: Load Training Data

Upload your `ability_training_data.jsonl` file to Colab using the file browser on the left.

In [ ]:
from datasets import load_dataset

# Load JSONL training data
dataset = load_dataset("json", data_files="ability_training_data.jsonl", split="train")

print(f"📊 Loaded {len(dataset)} training examples")
print("\nExample:")
print(dataset[0])

In [ ]:
# Format dataset for instruction fine-tuning (Qwen2.5 chat format)
def format_prompts(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Qwen2.5 chat template format
        text = f"""<|im_start|>system
You are a creative game designer for Lexmancer, an elemental combat game. Generate ability JSON exactly matching the schema.<|im_end|>
<|im_start|>user
{instruction}{f' {input_text}' if input_text else ''}<|im_end|>
<|im_start|>assistant
{output}<|im_end|>"""
        texts.append(text)
    
    return {"text": texts}

# Apply formatting
dataset = dataset.map(format_prompts, batched=True)

print("✅ Dataset formatted!")
print("\nFormatted example:")
print(dataset[0]["text"][:500] + "...")

## Step 5: Configure Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences
    args=TrainingArguments(
        # Training hyperparameters
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 2*4 = 8
        warmup_steps=10,
        num_train_epochs=3,  # Adjust based on dataset size
        # max_steps=100,  # Alternative to num_train_epochs
        learning_rate=2e-4,
        
        # Optimization
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        
        # Logging & checkpointing
        logging_steps=1,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        
        # Other
        seed=3407,
    ),
)

print("✅ Trainer configured!")

## Step 6: Train!

This will take 15-60 minutes depending on:
- Dataset size
- Number of epochs
- GPU type (T4 vs A100)

In [ ]:
# Show GPU memory before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU: {gpu_stats.name}")
print(f"Memory: {start_gpu_memory}GB / {max_memory}GB allocated\n")

# Start training
trainer_stats = trainer.train()

# Show final stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_training = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print(f"\n✅ Training complete!")
print(f"Peak memory: {used_memory}GB ({used_percentage}% of {max_memory}GB)")
print(f"Training used: {used_memory_for_training}GB")

## Step 7: Test Generation (Optional)

In [ ]:
# Enable fast inference mode
FastLanguageModel.for_inference(model)

# Test prompt
test_prompt = """<|im_start|>system
You are a creative game designer for Lexmancer, an elemental combat game. Generate ability JSON exactly matching the schema.<|im_end|>
<|im_start|>user
Create an ability for combining Shadow and Lightning elements.<|im_end|>
<|im_start|>assistant
"""

inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)

generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Extract assistant response
response = generated_text.split("<|im_start|>assistant")[-1].strip()
print("Generated Ability:")
print(response)

## Step 8: Save Model (Merged)

In [ ]:
# Save LoRA adapters only (small, ~50MB)
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

print("✅ LoRA adapters saved to 'lora_model/'")

In [ ]:
# Merge LoRA adapters with base model (creates full model)
model.save_pretrained_merged(
    "merged_model",
    tokenizer,
    save_method="merged_16bit",  # Options: merged_16bit, merged_4bit, lora
)

print("✅ Merged model saved to 'merged_model/'")

## Step 9: Convert to GGUF Format

This converts the model to GGUF format for use with LLamaSharp.

In [ ]:
# Install llama.cpp for conversion
%%capture
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make

In [ ]:
# Install dependencies for conversion
%%capture
!pip install gguf sentencepiece protobuf

In [ ]:
# Convert to FP16 GGUF (unquantized)
!python llama.cpp/convert_hf_to_gguf.py \
    merged_model \
    --outfile lexmancer-qwen2.5-7b-fp16.gguf \
    --outtype f16

print("✅ FP16 GGUF created!")

In [ ]:
# Quantize to Q4_K_M (same as your current model)
!./llama.cpp/llama-quantize \
    lexmancer-qwen2.5-7b-fp16.gguf \
    lexmancer-qwen2.5-7b-q4_k_m.gguf \
    Q4_K_M

print("✅ Q4_K_M GGUF created!")

## Step 10: Download Your Fine-Tuned Model

Download `lexmancer-qwen2.5-7b-q4_k_m.gguf` and replace:
```
Assets/LLM/qwen2.5-7b-instruct-q4_k_m.gguf
```

You may also want to download the FP16 version for further fine-tuning later.

In [ ]:
# Check file sizes
!ls -lh *.gguf

# Download files
from google.colab import files

print("\nDownloading Q4_K_M model (recommended for game)...")
files.download('lexmancer-qwen2.5-7b-q4_k_m.gguf')

# Uncomment to also download FP16 (much larger, ~15GB)
# print("\nDownloading FP16 model (for future fine-tuning)...")
# files.download('lexmancer-qwen2.5-7b-fp16.gguf')

## Alternative: Save to Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy to Drive
!mkdir -p "/content/drive/MyDrive/Lexmancer/Models"
!cp lexmancer-qwen2.5-7b-q4_k_m.gguf "/content/drive/MyDrive/Lexmancer/Models/"
!cp lexmancer-qwen2.5-7b-fp16.gguf "/content/drive/MyDrive/Lexmancer/Models/"

print("✅ Models saved to Google Drive!")

---

## Summary

**What we did:**
1. ✅ Loaded Qwen2.5-7B-Instruct base model
2. ✅ Added LoRA adapters for efficient fine-tuning
3. ✅ Fine-tuned on ability generation examples
4. ✅ Merged adapters back into base model
5. ✅ Converted to GGUF format
6. ✅ Quantized to Q4_K_M

**Next steps:**
1. Download `lexmancer-qwen2.5-7b-q4_k_m.gguf`
2. Replace `Assets/LLM/qwen2.5-7b-instruct-q4_k_m.gguf` in your project
3. Test in-game!

**Expected improvements:**
- Better JSON structure compliance
- More creative ability names/descriptions
- Better element synergy understanding
- Consistent damage/effect balancing